在Part 1中我们提到创建工具需要构建一个schema和python函数给到model进行工具的绑定，这是最底层的实现方式，LangChain给我们进行了封装，通过@tool装饰器，能够快速创建一个供Agent调用的工具。

# 基本用法

In [ ]:
# 导入装饰器
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """在客户数据库中搜索与查询条件匹配的记录。

    Args:
        query: 要查找的搜索关键词
        limit: 返回结果的最大数量
    """
    return f"为 '{query}' 找到 {limit} 个结果"

最佳实践：
1. 提供类型提示，该提示定义了工具的输入结构。
2. 提供文档字符串，并且信息应明确且简洁，以帮助模型理解工具的用途。

# @tool的参数

| 参数名 | 类型 | 说明 |
|--------|------|------|
| name_or_callable | str \| Callable \| None | 工具名称或可调用对象，可以是字符串标识符或函数 |
| runnable | Runnable \| None | 可选的 Runnable 对象 |
| description | str \| None | 工具的描述信息 |
| return_direct | bool | 是否直接返回结果，默认为 False |
| args_schema | ArgsSchema \| None | 参数模式，用于定义工具的输入结构 |
| infer_schema | bool | 是否自动推断输入模式，默认为 True |
| response_format | Literal["content", "content_and_artifact"] | 响应格式，可以是 "content" 或 "content_and_artifact" |
| parse_docstring | bool | 是否解析文档字符串，默认为 False |
| error_on_invalid_docstring | bool | 当文档字符串无效时是否报错，默认为 True |
| extras | dict[str, Any] \| None | 额外的元数据字典 |

下面我们介绍常用的几个：工具名称、工具描述和args_schema

## 工具名称

工具名称有别于函数名，是模型与工具之间通信的关键标识符，它直接影响 Agent 能否正确调用工具，应遵循唯一名称 + 功能的“语义标签”的定义原则。

In [ ]:
@tool(name_or_callable="web_search")  # 工具名称，只有工具名称一个参数的话，可以省略关键字name_or_callable
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

## 工具描述

In [ ]:
@tool(name_or_callable="web_search", description="该工具用于根据用户提供的关键词，在网络上检索最新或相关的信息。智能体可在需要实时数据、事实核查或补充知识时调用此工具")  # 新增工具描述
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

description和函数docstring的区别：

<div align=center><img src="https://techinsight.com.cn/course/3707820c-06a6-4e5e-8615-6b0aaea1b699.png" width=80%></div>

## args_schema

函数的参数是给python程序使用的，大模型是看不到需要这些参数的，因此需要定义args_schema来告诉大模型工具函数接收什么类型的参数。LangChain支持使用Pydantic模型或JSON Schema来定义复杂的args_schema。

使用Pydantic模型：

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

# 定义天气查询的输入模型
class WeatherInput(BaseModel):
    """天气查询的输入参数。"""
    location: str = Field(description="城市名称或坐标")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="温度单位偏好"
    )
    include_forecast: bool = Field(
        default=False,
        description="是否包含5天预报"
    )

# 使用自定义输入模型定义工具
@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """获取当前天气及可选的预报信息。"""
    temp = 22 if units == "celsius" else 72
    result = f"当前 {location} 的天气：{temp} 摄氏度"
    if include_forecast:
        result += "\n未来 5 天预报：晴天"
    return result

使用JSON Schema：

In [ ]:
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"]
}

@tool(args_schema=weather_schema)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """获取当前天气及可选的预报信息。"""
    temp = 22 if units == "celsius" else 72
    result = f"当前 {location} 的天气：{temp} 摄氏度"
    if include_forecast:
        result += "\n未来 5 天预报：晴天"
    return result

如果不指定args_schema，默认会根据函数的参数类型和注释来生成pydantic类型的args_schema。

In [10]:
# 导入装饰器
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """在客户数据库中搜索与查询条件匹配的记录。

    Args:
        query: 要查找的搜索关键词
        limit: 返回结果的最大数量
    """
    return f"为 '{query}' 找到 {limit} 个结果"

print(search_database)

name='search_database' description='在客户数据库中搜索与查询条件匹配的记录。\n\n    Args:\n        query: 要查找的搜索关键词\n        limit: 返回结果的最大数量' args_schema=<class 'langchain_core.utils.pydantic.search_database'> func=<function search_database at 0x106bd3420>
